# Aluno: Aléxandros Augustus

## 1 Classificação

### 1.1 Análise da base de dados e pré-processamento

In [ ]:
#importando ferramentas principais
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
%matplotlib inline

#importando ferramentas adicionais
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import neighbors
from sklearn import svm
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsRegressor
from sklearn import tree
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

In [ ]:
#carregando a base e visualizando-a
data = pd.read_csv('train-titanic.csv')

print(data.shape) #(linhas, colunas)
data.head() #5 primeiras instâncias

Detalhamento das colunas extraído do [Kaggle](https://www.kaggle.com/c/titanic/data) e traduzido:<br>
**Colunas:**<br>
*PassengerId:* ID único do passageiro;<br>
*Survived:* Sobreviveu (1) ou morreu (0);<br>
*Pclass:* Classe do passageiro (1ª, 2ª, ou 3ª);<br>
*Name:* Nome do passageiro;<br>
*Sex:* Sexo do passageiro;<br>
*Age:* Idade do passageiro;<br>
*SibSp:* Número de irmãos/cônjugues a bordo do Titanic;<br>
*Parch:* Número de pais/filhos a bordo do Titanic;<br>
*Ticket:* Número do bilhete;<br>
*Fare:* Valor pago pelo bilhete;<br>
*Cabin:* Número da cabine<br>
*Embarked:* Onde o passageiro embarcou no navio (C = Cherbourg, S = Southampton, Q = Queenstown)<br>
<br>
Para melhor classificação, serão eliminadas as colunas *PassengerId*, *Name* e *Ticket* pois apresentam características únicas aos passageiros que não contribuem para determinar sua sobrevivência. Além disso, é necessário converter os dados de *Sex* e *Embarked* em dados numéricos para que seja possível processá-los com o scikit-learn.

In [ ]:
#eliminação de colunas com dados pessoais ou únicos para cada passageiro
newdata = data.drop(columns=['PassengerId','Name','Ticket'])

#conversão de dados categóricos em dados numéricos
newdata['Sex'] = LabelEncoder().fit_transform(data['Sex'].astype('str'))
newdata['Embarked'] = LabelEncoder().fit_transform(data['Embarked'].astype('str'))

#visualização da nova base de dados
print(newdata.shape)
newdata.head()

É possível observar que a nova base agora possui 9 colunas em vez de 12, porém o número de linhas foi inalterado.<br>
A nova legenda para as colunas *Sex* e *Embarked* é a seguinte:<br>
*Sex:* female = 0, male = 1<br>
*Embarked:* C = 0, Q = 1, S = 2<br>
<br>
Para melhor acurácia é importante também verificar a existência de atributos incompletos e tratá-los adequadamente.

In [ ]:
#verifica quantas linhas em cada coluna possuem atributos com valor nulo
null_columns = newdata.columns[newdata.isnull().any()]
newdata[null_columns].isnull().sum()

A partir do código anterior verifica-se que a coluna *Age* possui cerca de 19,87% linhas inválidas, e a coluna *Cabin* possui cerca de 77,10% linhas inválidas.<br>
A estratégia que será adotada nesta situação será remover a coluna *Cabin*, pois mais de 70% desta não possui informação, a coluna *Age* será mantida como está, visto que menos de 30% da mesma se encontra comprometida.<br>
Após tratar todas as colunas, verifica-se a matriz de correlação.

In [ ]:
#removendo a coluna Cabin
newdata = newdata.drop(columns=['Cabin'])

#plotando a matriz de correlação
corr = newdata.corr()
corr.style.background_gradient(cmap='coolwarm')

A partir das informações presentes na matriz de correlação, serão removidas todas as colunas cujo módulo de correlação com a coluna *Survived* é menor que 25%, ou seja, todas as colunas cuja correlação com a coluna *Survived* seja superior a -0,25 para valores negativos, ou inferior a 0,25 para valores positivos.<br>
No caso serão removidas as colunas *Pclass*, *Age*, *SibSp*, *Parch* e *Embarked*.

In [ ]:
#remoção de colunas com baixa correlação com o atributo de saída
newdata = newdata.drop(columns=['Pclass','Age','SibSp','Parch','Embarked'])

#visualização da nova base
print(newdata.shape)
newdata.head()

Agora que a base possui apenas as colunas necessárias para inferir a classe *Survived*, verificaremos a presença de possíveis *outliers* nos atributos de entrada e o balanceamento das classes.

In [ ]:
#histograma das classes
newdata.hist('Survived')

In [ ]:
#histograma do atributo Sex
newdata.hist('Sex')

In [ ]:
#boxplot do atributo Fare
newdata.boxplot('Fare')

Através das plotagens feitas, é possível concluir que:<br>
As classes não estão perfeitamente balanceadas, a classe "Morreu" possui cerca de 500 instâncias, enquanto que a classe "Sobreviveu" possui cerca de 300 instâncias. Entretanto a diferença entre elas é pequena o suficiente para não apresentar um problema onde seria necessário remover instâncias de uma classe para que seja possível realizar predições mais precisas.<br>
Não há *outliers* no atributo "Sexo".<br>
O atributo "Tarifa" possui muitos *outliers*, porém, removeremos apenas aqueles muito mais distantes, cujo valor excede 300.

In [ ]:
#remove linhas cujo valor da tarifa ultrapassa 300
newdata = newdata.loc[newdata['Fare']<=300]

#verifica o número de linhas e colunas da nova base
newdata.shape

Agora a base possui apenas 888 linhas, em vez de 891, ou seja, 3 linhas foram excluídas.<br>
Com isso a base está pronta para ser dividida em conjuntos de treinamento e teste. Tais conjuntos serão gerados de forma aleatória, portanto, seguirá em anexo os conjuntos utilizados na execução deste programa, já que cada vez que o mesmo for executado, conjuntos diferentes serão gerados, assim fazendo com que resultados diferentes sejam encontrados.

In [ ]:
#Não execute esta célula a menos que queira gerar novos conjuntos de treinamento e teste

#Divide a base em treinamento e teste com o conjunto de teste correspondente a 20% da base original
train, test = train_test_split(newdata,test_size=0.2)

#Exporta os conjuntos de treinamento e teste gerados como arquivos .csv
#Comente esta linha caso não deseje exportar novos arquivos
export_csv = train.to_csv('classificacao-train.csv')
export_csv = test.to_csv('classificacao-test.csv')

### 1.2 Escolha dos métodos
Para esta atividade, foram escolhidos os métodos KNN e SVM.<br>
O método KNN foi escolhido pois é um método simples e de rápida execução que pode apresentar bons resultados apesar de sua simplicidade, já o método SVM foi escolhido pois é outro algoritmo simples de classificação voltado para casos em que deseja-se separar 2 classes, porém, utilizando uma técnica semelhante a agrupamentos para prever os resultados.

In [ ]:
#Caso deseje replicar os resultos obtidos, execute os comandos desta célula
train = pd.read_csv('classificacao-train.csv')
test = pd.read_csv('classificacao-test.csv')

#### 1.2.1 Implementação do KNN
Decidiu-se utilizar o método de KNN com parâmetro k=1 e k=3, escolhidos por serem os menores valores válidos de k.

In [ ]:
#definindo os classificadores para k=1 e k=3
knn1 = neighbors.KNeighborsClassifier(1)
knn3 = neighbors.KNeighborsClassifier(3)

#treinando os métodos e armazenando as predições em vetores
knn1.fit(train,train['Survived'])
result_knn1 = knn1.predict(test)

knn3.fit(train,train['Survived'])
result_knn3 = knn3.predict(test)

#### 1.2.2 Implementação do SVM
Optou-se por utilizar o método de SVM com funções kernel "linear" e "polinomal de grau 3". A função linear foi escolhida por ser a mais simples do SVM, enquanto que a polinomial é quase tão simples quanto a linear, afinal a função linear é uma função polinomial de grau 1, o grau 3 foi escolhido por apresentar curvatura mais semelhante à liear que a polinomial de grau 2.

In [ ]:
#definindo os classificadores para kernel linear e polinomial
svmline = svm.SVC(gamma='auto',kernel='linear')
svmpoly = svm.SVC(gamma='auto',kernel='poly')

#treinando os métodos e armazenando as predições em vetores
svmline.fit(train,train['Survived'])
result_svmline = svmline.predict(test)

svmpoly.fit(train,train['Survived'])
result_svmpoly = svmpoly.predict(test)

### 1.3 Resultados
Para esta atividade, considerou-se a classe negativa como sendo a classe "Morreu" (0) e a classe positiva como sendo a classe "Sobreviveu" (1)
#### 1.3.1 Resultados KNN com k=1

In [ ]:
#geração de relatório dos testes
cm = confusion_matrix(test['Survived'],result_knn1)
print ('Matriz de Confusão :')
print(cm,'\n')
print ('Métricas : ')
print (classification_report(test['Survived'],result_knn1)) 

**Verdadeiros Negativos:** 117<br>
**Falsos Negativos:** 2<br>
**Falsos Positivos:** 1<br>
**Verdadeiros Positivos:** 58<br>
**Acurácia:** 98%<br>
**Precisão:** 97%<br>
**Especificidade:** 98%<br>
**Sensibilidade:** 98%<br>
#### 1.3.2 Resultados KNN com k=3

In [ ]:
#geração de relatório dos testes
cm = confusion_matrix(test['Survived'],result_knn3)
print ('Matriz de Confusão :')
print(cm,'\n')
print ('Métricas : ')
print (classification_report(test['Survived'],result_knn3)) 

**Verdadeiros Negativos:** 112<br>
**Falsos Negativos:** 7<br>
**Falsos Positivos:** 1<br>
**Verdadeiros Positivos:** 58<br>
**Acurácia:** 96%<br>
**Precisão:** 89%<br>
**Especificidade:** 94%<br>
**Sensibilidade:** 98%<br>
#### 1.3.3 Resultados SVM com função de kernel linear

In [ ]:
#geração de relatório dos testes
cm = confusion_matrix(test['Survived'],result_svmline)
print ('Matriz de Confusão :')
print(cm,'\n')
print ('Métricas : ')
print (classification_report(test['Survived'],result_svmline)) 

**Verdadeiros Negativos:** 119<br>
**Falsos Negativos:** 0<br>
**Falsos Positivos:** 0<br>
**Verdadeiros Positivos:** 59<br>
**Acurácia:** 100%<br>
**Precisão:** 100%<br>
**Especificidade:** 100%<br>
**Sensibilidade:** 100%<br>
#### 1.3.4 Resultados SVM com função de kernel polinomial de grau 3

In [ ]:
#geração de relatório dos testes
cm = confusion_matrix(test['Survived'],result_svmline)
print ('Matriz de Confusão :')
print(cm,'\n')
print ('Métricas : ')
print (classification_report(test['Survived'],result_svmline)) 

**Verdadeiros Negativos:** 119<br>
**Falsos Negativos:** 0<br>
**Falsos Positivos:** 0<br>
**Verdadeiros Positivos:** 59<br>
**Acurácia:** 100%<br>
**Precisão:** 100%<br>
**Especificidade:** 100%<br>
**Sensibilidade:** 100%<br>
#### 1.3.5 Discussão dos resultados obtidos
Verifica-se que o algoritmo que apresentou os melhores resultados para este conjunto específico de treinamento e teste foi o SVM, tendo acurácia de 100% tanto para função kernel linear quanto para a função polinomial de grau 3.<br>
A acurácia representa a porcentagem de instâncias classificadas corretamente dentre todas, por isso uma acurácia de 100% significa que todas as outras métricas também possuem valor 100%, já que o número de instâncias corretamente classificadas é igual ao número total de instâncias no universo, tanto considerando-se a classe positiva quanto a negativa.<br>
A precisão representa a porcentagem de instâncias corretamente classificadas positivas dentre todas as instâncias classificadas positivas.<br>
A especificidade representa a porcentagem de instâncias corretamente classificadas negativas dentre todas as instâncias que pertencem à classe negativa. No relatório de classificação é representada pelo "recall" da classe negativa.<br>
A sensibilidade representa a porcentagem de instâncias corretamente classificadas positivas dentre todas as instâncias que pertencem à classe positiva. No relatório de classificação é representada pelo "recall" da classe positiva.<br>
Outro ponto importante a ser levantado é o fato do algortimo KNN com k=3 ter obtido uma acurácia menor que KNN com k=1. Uma possível explicação para tal fenômeno é que as instâncias que foram corretamente classificadas para k=1 e incorretamente classificadas para k=3 estavam muito próximas da fronteira de decisão, e ao ser ampliado o número de vizinhos verificados para classificá-las, foram encontrados mais vizinhos da classe incorreta que da classe correta, porém o vizinho mais próximo de tais instâncias ainda era aquele da classe correta.
## 2 Regressão
### 2.1 Análise da base de dados e pré-processamento

In [ ]:
#carregando uma nova base e visualizando-a
data = pd.read_csv('train.csv')

print(data.shape) #(linhas, colunas)
data.head()

É possível notar que há muitas colunas na base, e muitas delas possuem atributos não-numéricos, por isso vamos discretizar a base e em seguida analizar sua matriz de correlação para eliminar atributos que tenham módulo de correlação com a coluna *SalePrice* - que representa a saída que desejamos prever - inferior a 30%.<br>
Para selecionar as colunas que necessitam de discretização, verificou-se no [Kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data) as colunas com valores do tipo *string*.

In [ ]:
#inicialização de uma nova base igual à base de dados original
newdata = data

#discretização de colunas
newdata['MSZoning'] = LabelEncoder().fit_transform(newdata['MSZoning'].astype('str'))
newdata['LotFrontage'] = LabelEncoder().fit_transform(newdata['LotFrontage'].astype('str'))
newdata['Street'] = LabelEncoder().fit_transform(newdata['Street'].astype('str'))
newdata['Alley'] = LabelEncoder().fit_transform(newdata['Alley'].astype('str'))
newdata['LotShape'] = LabelEncoder().fit_transform(newdata['LotShape'].astype('str'))
newdata['LandContour'] = LabelEncoder().fit_transform(newdata['LandContour'].astype('str'))
newdata['Utilities'] = LabelEncoder().fit_transform(newdata['Utilities'].astype('str'))
newdata['LotConfig'] = LabelEncoder().fit_transform(newdata['LotConfig'].astype('str'))
newdata['LandSlope'] = LabelEncoder().fit_transform(newdata['LandSlope'].astype('str'))
newdata['Neighborhood'] = LabelEncoder().fit_transform(newdata['Neighborhood'].astype('str'))
newdata['Condition1'] = LabelEncoder().fit_transform(newdata['Condition1'].astype('str'))
newdata['Condition2'] = LabelEncoder().fit_transform(newdata['Condition2'].astype('str'))
newdata['BldgType'] = LabelEncoder().fit_transform(newdata['BldgType'].astype('str'))
newdata['HouseStyle'] = LabelEncoder().fit_transform(newdata['HouseStyle'].astype('str'))
newdata['RoofStyle'] = LabelEncoder().fit_transform(newdata['RoofStyle'].astype('str'))
newdata['RoofMatl'] = LabelEncoder().fit_transform(newdata['RoofMatl'].astype('str'))
newdata['Exterior1st'] = LabelEncoder().fit_transform(newdata['Exterior1st'].astype('str'))
newdata['Exterior2nd'] = LabelEncoder().fit_transform(newdata['Exterior2nd'].astype('str'))
newdata['MasVnrType'] = LabelEncoder().fit_transform(newdata['MasVnrType'].astype('str'))
newdata['MasVnrArea'] = LabelEncoder().fit_transform(newdata['MasVnrArea'].astype('str'))
newdata['ExterQual'] = LabelEncoder().fit_transform(newdata['ExterQual'].astype('str'))
newdata['ExterCond'] = LabelEncoder().fit_transform(newdata['ExterCond'].astype('str'))
newdata['Foundation'] = LabelEncoder().fit_transform(newdata['Foundation'].astype('str'))
newdata['BsmtQual'] = LabelEncoder().fit_transform(newdata['BsmtQual'].astype('str'))
newdata['BsmtCond'] = LabelEncoder().fit_transform(newdata['BsmtCond'].astype('str'))
newdata['BsmtExposure'] = LabelEncoder().fit_transform(newdata['BsmtExposure'].astype('str'))
newdata['BsmtFinType1'] = LabelEncoder().fit_transform(newdata['BsmtFinType1'].astype('str'))
newdata['BsmtFinType2'] = LabelEncoder().fit_transform(newdata['BsmtFinType2'].astype('str'))
newdata['Heating'] = LabelEncoder().fit_transform(newdata['Heating'].astype('str'))
newdata['HeatingQC'] = LabelEncoder().fit_transform(newdata['HeatingQC'].astype('str'))
newdata['CentralAir'] = LabelEncoder().fit_transform(newdata['CentralAir'].astype('str'))
newdata['Electrical'] = LabelEncoder().fit_transform(newdata['Electrical'].astype('str'))
newdata['KitchenQual'] = LabelEncoder().fit_transform(newdata['KitchenQual'].astype('str'))
newdata['Functional'] = LabelEncoder().fit_transform(newdata['Functional'].astype('str'))
newdata['FireplaceQu'] = LabelEncoder().fit_transform(newdata['FireplaceQu'].astype('str'))
newdata['GarageType'] = LabelEncoder().fit_transform(newdata['GarageType'].astype('str'))
newdata['GarageYrBlt'] = LabelEncoder().fit_transform(newdata['GarageYrBlt'].astype('str'))
newdata['GarageFinish'] = LabelEncoder().fit_transform(newdata['GarageFinish'].astype('str'))
newdata['GarageQual'] = LabelEncoder().fit_transform(newdata['GarageQual'].astype('str'))
newdata['GarageCond'] = LabelEncoder().fit_transform(newdata['GarageCond'].astype('str'))
newdata['PavedDrive'] = LabelEncoder().fit_transform(newdata['PavedDrive'].astype('str'))
newdata['PoolQC'] = LabelEncoder().fit_transform(newdata['PoolQC'].astype('str'))
newdata['Fence'] = LabelEncoder().fit_transform(newdata['Fence'].astype('str'))
newdata['MiscFeature'] = LabelEncoder().fit_transform(newdata['MiscFeature'].astype('str'))
newdata['SaleType'] = LabelEncoder().fit_transform(newdata['SaleType'].astype('str'))
newdata['SaleCondition'] = LabelEncoder().fit_transform(newdata['SaleCondition'].astype('str'))

#visualização da nova base
newdata.head()

In [ ]:
#geração da matriz de correlação
corr = newdata.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
#remoção de colunas
newdata = newdata.drop(columns=['Id','MSSubClass','MSZoning','LotFrontage','LotArea','Street','Alley','LotShape',
                               'LandContour','Utilities','LotConfig','LandSlope','Neighborhood','Condition1','Condition2',
                               'BldgType','HouseStyle','OverallCond','RoofStyle','RoofMatl','Exterior1st','Exterior2nd',
                               'MasVnrType','MasVnrArea','ExterQual','ExterCond','BsmtCond','BsmtFinType1','BsmtFinType2',
                               'BsmtFinSF2','BsmtUnfSF','Heating','CentralAir','Electrical','LowQualFinSF','BsmtFullBath',
                               'BsmtHalfBath','HalfBath','BedroomAbvGr','KitchenAbvGr','Functional','GarageQual',
                                'GarageCond','PavedDrive','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','PoolQC',
                               'Fence','MiscFeature','MiscVal','MoSold','YrSold','SaleType','SaleCondition'])

print(newdata.shape)
newdata.head()

Em seguida, é recomendável verificar a existência de linhas com falta de dados ou dados inválidos.

In [ ]:
#apresenta número de instâncias com valores inválidos por coluna
null_columns = newdata.columns[newdata.isnull().any()]
newdata[null_columns].isnull().sum()

Constata-se que todas as instâncias presentes na nova base de dados possuem valores válidos em todas as suas colunas.<br>
Vamos então plotar os boxplots de cada coluna e remover linhas com *outliers* muito distantes.

In [ ]:
newdata.boxplot('SalePrice')

In [ ]:
#remove outliers mais distantes
newdata = newdata.loc[newdata['SalePrice']<=700000]

#exibe o novo tamanho da base
newdata.shape

In [ ]:
newdata.boxplot('OverallQual')

In [ ]:
newdata.boxplot('YearBuilt')

In [ ]:
newdata.boxplot('YearRemodAdd')

In [ ]:
newdata.boxplot('Foundation')

In [ ]:
newdata.boxplot('BsmtQual')

In [ ]:
newdata.boxplot('BsmtExposure')

In [ ]:
newdata.boxplot('BsmtFinSF1')

In [ ]:
newdata = newdata.loc[newdata['BsmtFinSF1']<=5000]

newdata.shape

In [ ]:
newdata.boxplot('TotalBsmtSF')

In [ ]:
newdata.boxplot('HeatingQC')

In [ ]:
newdata.boxplot('1stFlrSF')

In [ ]:
newdata.boxplot('2ndFlrSF')

In [ ]:
newdata.boxplot('GrLivArea')

In [ ]:
newdata = newdata.loc[newdata['GrLivArea']<=4000]

newdata.shape

In [ ]:
newdata.boxplot('FullBath')

In [ ]:
newdata.boxplot('KitchenQual')

In [ ]:
newdata.boxplot('TotRmsAbvGrd')

In [ ]:
newdata = newdata.loc[newdata['TotRmsAbvGrd']<=12]

newdata.shape

In [ ]:
newdata.boxplot('Fireplaces')

In [ ]:
newdata.boxplot('FireplaceQu')

In [ ]:
newdata.boxplot('GarageType')

In [ ]:
newdata.boxplot('GarageYrBlt')

In [ ]:
newdata.boxplot('GarageFinish')

In [ ]:
newdata.boxplot('GarageCars')

In [ ]:
newdata.boxplot('GarageArea')

In [ ]:
newdata.boxplot('WoodDeckSF')

In [ ]:
newdata = newdata.loc[newdata['WoodDeckSF']<=800]

newdata.shape

In [ ]:
newdata.boxplot('OpenPorchSF')

Temos então que o número de linhas da nova base de dados foi reduzido de 1460 para 1454, ou seja, 6 instâncias foram eliminadas por conterem valores muito distantes da maioria.<br>
Para que possamos obter resultados ainda melhores, normalizaremos a base antes de aplicar os métodos de aprendizagem de máquina. Em seguida, a base normalizada será dividida em conjuntos de treinamento e teste para que possam ser aplicados os métodos de aprendizagem de máquina.<br>
Os conjuntos de treinamento e teste que terão seus resultados analisados neste relatório seguirão em anexo com o mesmo, visto que tais conjuntos estão sendo gerados de forma aleatória, e a cada execução conjuntos diferentes são gerados de forma que resultados diferentes são obtidos.

In [ ]:
#limita os valores de todas as colunas em um alcance de 0 a 10
normaldata=((newdata-newdata.min())/(newdata.max()-newdata.min()))*10

#exibe a base normalizada
normaldata.head()

In [ ]:
#Não execute esta célula a menos que queira gerar novos conjuntos de treinamento e teste

#Divide a base em treinamento e teste com o conjunto de teste correspondente a 20% da base original
train, test = train_test_split(normaldata,test_size=0.2)

#Exporta os conjuntos de treinamento e teste gerados como arquivos .csv
#Comente esta linha caso não deseje exportar novos arquivos
export_csv = train.to_csv('regressao-train.csv')
export_csv = test.to_csv('regressao-test.csv')

### 2.2 Escolha dos métodos
Para este problema, foram escolhidos os métodos KNN e árvore de decisão.<br>
O método de KNN foi escolhido por ser um dos mais simples e, mesmo com sua simplicidade, ser capaz de gerar bons resultados. O valor de k escolhido foi k=2 pois assim o resultado previsto será a média das duas instâncias mais próximas das quais se tem conhecimento.<br>
Já a árvore de decisão foi escolhida por ser um método que geralmente garante bons resultados mesmo em problemas de regressão, sendo capaz de gerar uma boa aproximação linear dos valores reais.

In [ ]:
#caso deseje replicar os resultados obtidos, execute os comandos desta célula
train = pd.read_csv('regressao-train.csv')
test = pd.read_csv('regressao-test.csv')

#### 2.2.1 Implementação do KNN para regressão

In [ ]:
#definindo o classificador para k=2
knnr = KNeighborsRegressor(n_neighbors=2)

#treinando o método e armazenando as predições em um vetor
knnr.fit(train,train['SalePrice'])
result_knnr = knnr.predict(test)

#### 2.2.2 Implementação da árvore de decisão para regressão

In [ ]:
#definindo o classificador da árvore de decisão
dtr = tree.DecisionTreeRegressor()

#treinando o método e armazenando as predições em um vetor
dtr.fit(train,train['SalePrice'])
result_dtr = dtr.predict(test)

### 2.3 Resultados
Para análise dos resultados utilizou-se como métricas as taxas de erro médio absoluta e quadrática.<br>
A taxa de erro médio absoluta é a média do somatório das diferenças dos valores reais das instâncias com os valores previstos para as mesmas.<br>
Já a taxa de erro média quadrática é semelhante à absoluta, com o diferencial que ao invés de usar as o somatório das diferenças, utiliza-se o somatório das diferenças elevadas ao quadrado, dessa forma caso a diferença seja muito grande a taxa cresce em um ritmo exponencial.<br>
O menor valor possível para estas taxas é 0, significando que 100% das instâncias foram previstas perfeitamente. Como a base de dados foi normalizada esses valores passam a ser proporcionais ao número máximo para o qual a base é normalizada. Ao normalizar a base para o valor 10 foi possível encontrar o menor valor possível para as taxas de erro na ordem de $10^{-1}$ para o método de KNN e na ordem de $10^{-2}$ para o erro médio absoluto da árvore de decisão e $10^{-3}$ para o erro médio quadrático da mesma.
#### 2.3.1 Resultados KNN para regressão com k=2

In [ ]:
print('Taxa de Erro Médio Absoluto: ',mean_absolute_error(test['SalePrice'], result_knnr))
print('Taxa de Erro Médio Quadrático: ',mean_squared_error(test['SalePrice'],result_knnr))

#### 2.3.2 Resultados Árvore de Decisão para regressão

In [ ]:
print('Taxa de Erro Médio Absoluto: ',mean_absolute_error(test['SalePrice'], result_dtr))
print('Taxa de Erro Médio Quadrático: ',mean_squared_error(test['SalePrice'],result_dtr))

## 3 Clusterização
### 3.1 Análise dos dados e preparação para uso de clusterização

In [ ]:
#carregando uma nova base e visualizando-a
data = pd.read_csv('transfer.csv')

print(data.shape) #(linhas, colunas)
data.head()

Pelo que se pode observar e pela descrição da base no [Kaggle](https://www.kaggle.com/thesiff/premierleague1819), sabe-se que a coluna *team* é a única que possui atributos não-numéricos, sendo assim vamos primeiramente gerar uma nova base com esse atributos discretizados.

In [ ]:
#criando nova base igual à original e discretizando-a
newdata = data

newdata['team'] = LabelEncoder().fit_transform(newdata['team'].astype('str'))

#exibindo nova base
newdata.head()

Para utilizar os métodos de clusterização e plotá-los posteriormente, é ideal que a base tenha sua dimensão, ou seja, seu número de colunas, reduzido para 2. Como a coluna *team* identifica diferentes times, esta será mantida. Para determinar o que fazer com as demais colunas vamos plotar a matrize de correlação para verificar se existem colunas com forte correlação que podem ser removidas.

In [ ]:
corr = newdata.corr()
corr.style.background_gradient(cmap='coolwarm')

Da matriz de correlação tem-se que a coluna *Total* é a que mais possui correlação acima de 50% com as demais colunas, isto se deve ao fato da coluna *Total* ser o somatório dos valores das colunas *end_2019* a *end_2010*, por este motivo esta coluna será a coluna a ser mantida na base final.

In [ ]:
#Removendo colunas desnecessárias
newdata = newdata.drop(columns=['end_2019','end_2018','end_2017','end_2016','end_2015','end_2014','end_2013','end_2012',
                               'end_2011','end_2010'])

#visualizando a base final
newdata.head()

Agora que temos a base final que será utilizada pelos algoritmos de clusterização e plotada, vamos gerar outra base semelhante a esta, mas com os valores da coluna *Total* para que possamos comparar os resultados da clusterização e plotagem em cada base.

In [ ]:
#cria e limita os valores da coluna Total a um alcance de 0 a 10 e exibe a base normalizada
normaldata['team'] = newdata['team']
normaldata['Total']=((newdata['Total']-newdata['Total'].min())/(newdata['Total'].max()-newdata['Total'].min()))*10

normaldata.head()

Para verificar como a normalização pode ter alterado a base, vamos gerar e comparar os boxplots da coluna *Total* em ambas as bases.

In [ ]:
newdata.boxplot('Total')

In [ ]:
normaldata.boxplot('Total')

Verifac-se então que a normalização apenas muda a escala do eixo y no boxplot, não alterando de nenhuma forma  ainformação dos dados, porém, tal mudança de escala pode aproximar mais certas instâncias, gerando diferentes agrupamentos para o mesmo método de clusterização dependendo de qual base seja usada. Isto será verificado ao final desta seção.
### 3.2 Implementação dos métodos
Apesar da proposta de utilizar valores de k iguais a 2, 5, 10 e 100, como não há mais que 20 instâncias qualquer valor de k acima de 20 é rejeitado pelos algoritmos, por isso os valores de k que serão utilizados são 2, 5 e 10.
#### 3.2.1 K-means
Para k=2 utilizou-se os valores 1, 10 e 100 para o número máximo de iterações, para os demais valores de k o número máximo de iterações é o padrão do scikit-learn (300).

In [ ]:
#definindo clusterizadores para bases normalizada e não-normalizada
km2_1 = KMeans(n_clusters=2,max_iter=1).fit(newdata)
kmn2_1 = KMeans(n_clusters=2,max_iter=1).fit(normaldata)
km2_10 = KMeans(n_clusters=2,max_iter=10).fit(newdata)
kmn2_10 = KMeans(n_clusters=2,max_iter=10).fit(normaldata)
km2_100 = KMeans(n_clusters=2,max_iter=100).fit(newdata)
kmn2_100 = KMeans(n_clusters=2,max_iter=100).fit(normaldata)
km5 = KMeans(n_clusters=5).fit(newdata)
kmn5 = KMeans(n_clusters=5).fit(normaldata)
km10 = KMeans(n_clusters=10).fit(newdata)
kmn10 = KMeans(n_clusters=10).fit(normaldata)

#criando funções para imprimir os resultados
def result_km2_1():
    print('K-means com 2 classes e máximo de 1 iteração:\n')
    print('Dados não-normalizados:\n')
    print('Classes: ',km2_1.labels_)
    print('Centróides:\n',km2_1.cluster_centers_)
    print('\n')
    print('Dados normalizados:\n')
    print('Classes: ',kmn2_1.labels_)
    print('Centróides:\n',kmn2_1.cluster_centers_)

def result_km2_10():
    print('K-means com 2 classes e máximo de 10 iterações:\n')
    print('Dados não-normalizados:\n')
    print('Classes: ',km2_10.labels_)
    print('Centróides:\n',km2_10.cluster_centers_)
    print('\n')
    print('Dados normalizados:\n')
    print('Classes: ',kmn2_10.labels_)
    print('Centróides:\n',kmn2_10.cluster_centers_)
    
def result_km2_100():
    print('K-means com 2 classes e máximo de 100 iterações:\n')
    print('Dados não-normalizados:\n')
    print('Classes: ',km2_100.labels_)
    print('Centróides:\n',km2_100.cluster_centers_)
    print('\n')
    print('Dados normalizados:\n')
    print('Classes: ',kmn2_100.labels_)
    print('Centróides:\n',kmn2_100.cluster_centers_)
    
def result_km5():
    print('K-means com 5 classes:\n')
    print('Dados não-normalizados:\n')
    print('Classes: ',km5.labels_)
    print('Centróides:\n',km5.cluster_centers_)
    print('\n')
    print('Dados normalizados:\n')
    print('Classes: ',kmn5.labels_)
    print('Centróides:\n',kmn5.cluster_centers_)
    
def result_km10():
    print('K-means com 10 classes:\n')
    print('Dados não-normalizados:\n')
    print('Classes: ',km10.labels_)
    print('Centróides:\n',km10.cluster_centers_)
    print('\n')
    print('Dados normalizados:\n')
    print('Classes: ',kmn10.labels_)
    print('Centróides:\n',kmn10.cluster_centers_)

#### 3.2.2 Hierárquico
Para k=2 utilizou-se os parâmetros de linkage *ward* (minimiza a variância dos clusters), *complete* (utiliza a maior distância), *average* (utiliza a distância média) e *single* (utiliza a menor distância). Para os demais valores de k o parâmetro utilizado foi o padrão do scikit-learn (*ward*). O parâmetro de linkage é o que determina a métrica utilizada pelo algoritmo para ajustar as classes.

In [ ]:
#definindo clusterizadores para bases normalizada e não-normalizada
h2_w = AgglomerativeClustering(n_clusters=2,linkage="ward").fit(newdata)
hn2_w = AgglomerativeClustering(n_clusters=2,linkage="ward").fit(normaldata)
h2_c = AgglomerativeClustering(n_clusters=2,linkage="complete").fit(newdata)
hn2_c = AgglomerativeClustering(n_clusters=2,linkage="complete").fit(normaldata)
h2_a = AgglomerativeClustering(n_clusters=2,linkage="average").fit(newdata)
hn2_a = AgglomerativeClustering(n_clusters=2,linkage="average").fit(normaldata)
h2_s = AgglomerativeClustering(n_clusters=2,linkage="single").fit(newdata)
hn2_s = AgglomerativeClustering(n_clusters=2,linkage="single").fit(normaldata)
h5 = AgglomerativeClustering(n_clusters=5).fit(newdata)
hn5 = AgglomerativeClustering(n_clusters=5).fit(normaldata)
h10 = AgglomerativeClustering(n_clusters=10).fit(newdata)
hn10 = AgglomerativeClustering(n_clusters=10).fit(normaldata)

#criando funções para imprimir os resultados
def result_h2_w():
    print('Hierárquico com 2 classes e variância:\n')
    print('Dados não-normalizados:\n')
    print('Classes: ',h2_w.labels_)
    print('\n')
    print('Dados normalizados:\n')
    print('Classes: ',hn2_w.labels_)
    
def result_h2_c():
    print('Hierárquico com 2 classes e maior distância:\n')
    print('Dados não-normalizados:\n')
    print('Classes: ',h2_c.labels_)
    print('\n')
    print('Dados normalizados:\n')
    print('Classes: ',hn2_c.labels_)
    
def result_h2_a():
    print('Hierárquico com 2 classes e distância média:\n')
    print('Dados não-normalizados:\n')
    print('Classes: ',h2_a.labels_)
    print('\n')
    print('Dados normalizados:\n')
    print('Classes: ',hn2_a.labels_)
    
def result_h2_s():
    print('Hierárquico com 2 classes e menor distância:\n')
    print('Dados não-normalizados:\n')
    print('Classes: ',h2_s.labels_)
    print('\n')
    print('Dados normalizados:\n')
    print('Classes: ',hn2_s.labels_)
    
def result_h5():
    print('Hierárquico com 5 classes:\n')
    print('Dados não-normalizados:\n')
    print('Classes: ',h5.labels_)
    print('\n')
    print('Dados normalizados:\n')
    print('Classes: ',hn5.labels_)
    
def result_h10():
    print('Hierárquico com 10 classes:\n')
    print('Dados não-normalizados:\n')
    print('Classes: ',h10.labels_)
    print('\n')
    print('Dados normalizados:\n')
    print('Classes: ',hn10.labels_)

### 3.3 Resultados
Primeiro vamos imprimir os relatórios dos algoritmos de clusterização e verificar se normalizar a base teve algum impacto dentro de cada algoritmo.

In [ ]:
result_km2_1()
result_km2_10()
result_km2_100()
result_km5()
result_km10()
result_h2_w()
result_h2_c()
result_h2_a()
result_h2_s()
result_h5()
result_h10()

É possível perceber de imediato que a base normalizada gerou grupos diferentes da base não-normalizada, mesmo que a informação contida nas bases tenha sido a mesma como verificado previamente nos boxplots.<br>
A seguir plotaremos os gráficos dos diferentes agrupamentos obtidos.

In [ ]:
plt.title('K-means com 2 classes e máximo de 1 iteração não-normalizado')
plt.scatter(newdata['team'],newdata['Total'],c=km2_1.labels_)
plt.show()

In [ ]:
plt.title('K-means com 2 classes e máximo de 1 iteração normalizado')
plt.scatter(normaldata['team'],normaldata['Total'],c=kmn2_1.labels_)
plt.show()

In [ ]:
plt.title('K-means com 2 classes e máximo de 10 iterações não-normalizado')
plt.scatter(newdata['team'],newdata['Total'],c=km2_10.labels_)
plt.show()

In [ ]:
plt.title('K-means com 2 classes e máximo de 10 iterações normalizado')
plt.scatter(normaldata['team'],normaldata['Total'],c=kmn2_10.labels_)
plt.show()

In [ ]:
plt.title('K-means com 2 classes e máximo de 100 iterações não-normalizado')
plt.scatter(newdata['team'],newdata['Total'],c=km2_100.labels_)
plt.show()

In [ ]:
plt.title('K-means com 2 classes e máximo de 100 iterações normalizado')
plt.scatter(normaldata['team'],normaldata['Total'],c=kmn2_100.labels_)
plt.show()

In [ ]:
plt.title('K-means com 5 classes não-normalizado')
plt.scatter(newdata['team'],newdata['Total'],c=km5.labels_)
plt.show()

In [ ]:
plt.title('K-means com 5 classes normalizado')
plt.scatter(normaldata['team'],normaldata['Total'],c=kmn5.labels_)
plt.show()

In [ ]:
plt.title('K-means com 10 classes não-normalizado')
plt.scatter(newdata['team'],newdata['Total'],c=km10.labels_)
plt.show()

In [ ]:
plt.title('K-means com 10 classes normalizado')
plt.scatter(normaldata['team'],normaldata['Total'],c=kmn10.labels_)
plt.show()

In [ ]:
plt.title('Hierárquico com 2 classes e linkage ward não-normalizado')
plt.scatter(newdata['team'],newdata['Total'],c=h2_w.labels_)
plt.show()

In [ ]:
plt.title('Hierárquico com 2 classes e linkage ward normalizado')
plt.scatter(normaldata['team'],normaldata['Total'],c=hn2_w.labels_)
plt.show()

In [ ]:
plt.title('Hierárquico com 2 classes e linkage complete não-normalizado')
plt.scatter(newdata['team'],newdata['Total'],c=h2_c.labels_)
plt.show()

In [ ]:
plt.title('Hierárquico com 2 classes e linkage complete normalizado')
plt.scatter(normaldata['team'],normaldata['Total'],c=hn2_c.labels_)
plt.show()

In [ ]:
plt.title('Hierárquico com 2 classes e linkage average não-normalizado')
plt.scatter(newdata['team'],newdata['Total'],c=h2_a.labels_)
plt.show()

In [ ]:
plt.title('Hierárquico com 2 classes e linkage average normalizado')
plt.scatter(normaldata['team'],normaldata['Total'],c=hn2_a.labels_)
plt.show()

In [ ]:
plt.title('Hierárquico com 2 classes e linkage single não-normalizado')
plt.scatter(newdata['team'],newdata['Total'],c=h2_s.labels_)
plt.show()

In [ ]:
plt.title('Hierárquico com 2 classes e linkage single normalizado')
plt.scatter(normaldata['team'],normaldata['Total'],c=hn2_s.labels_)
plt.show()

In [ ]:
plt.title('Hierárquico com 5 classes não-normalizado')
plt.scatter(newdata['team'],newdata['Total'],c=h5.labels_)
plt.show()

In [ ]:
plt.title('Hierárquico com 5 classes normalizado')
plt.scatter(normaldata['team'],normaldata['Total'],c=hn5.labels_)
plt.show()

In [ ]:
plt.title('Hierárquico com 10 classes não-normalizado')
plt.scatter(newdata['team'],newdata['Total'],c=h10.labels_)
plt.show()

In [ ]:
plt.title('Hierárquico com 10 classes normalizado')
plt.scatter(normaldata['team'],normaldata['Total'],c=h10.labels_)
plt.show()

A partir dos gráficos acima é possível perceber que para k=2 o algoritmo K-means não apresentou muita mudança variando-se o número máximo de iterações, porém, a base normalizada e a base não normalizada apresentaram resultados diferentes em ambos algoritmos e para todos os valores de k. Segundo um julgamento visual é possível afirmar que para valores de k superiores a 2 a base normalizada obteve resultados que parecem descrever melhor os agrupamentos enquanto que a base não-normalizada gerou grupos muito espalhados.<br>
Quanto ao algoritmo hierárquico para k=2, nota-se que não há grande distinção entre o linkage *ward* e *complete*, enquanto que os linkages *average* e *single* geraram resultados diferenciados.<br>
A partir de uma análise visual, é possível dizer que a escolha entre K-means ou hierárquico, no caso desta base, não faz muita diferença contanto que o parâmetro de linkage usado seja *ward* ou *complete*. E os melhores valores de k seriam 2 ou 5, visto que k=10 gerou gráficos muito confusos com pouca distinção entre as classes, entretanto, no caso de desejar-se utilizar k=10, o K-means normalizado apresentou resultados mais claros que o hierárquico, visualmente. Além disso vale ressaltar que utilizar a base normalizada gerou grupos mais organizados ou com melhor distribuição das instâncias que a base não-normalizada.<br>